# Access Azure resources from an online endpoint with a system-assigned managed identity

Additional packages are required for this example:

* Microsoft Azure Storage Management Client 
* Microsoft Azure Authorization Management Client

Install them with the following code:

In [ ]:
%pip install --pre azure-mgmt-storage
%pip install --pre azure-mgmt-authorization

## 1. Configure variables

### 1.1 Assign variables for the workspace and deployment

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace_name = "<AML_WORKSPACE_NAME>"
endpoint_name = "<ENDPOINT_NAME>"

### 1.2 Specify the storage details and file to access
A text file is located in the `managed-identities` folder called `hello.txt`. 

In [ ]:
storage_account_name = "<STORAGE_ACCOUNT_NAME>"
storage_container_name = "<CONTAINER_TO_ACCESS>"
file_name = "<FILE_TO_ACCESS>"

In [ ]:
import random

rand = random.randint(0, 10000)

endpoint_name = f"endpt-moe-{rand}"
storage_account_name = f"oepstorage{rand}"
storage_container_name = f"hellocontainer"
file_name = "hello.txt"

### 1.3 Retrieve the workspace location:

In [ ]:
from azure.ai.ml import MLClient
from azure.identity import AzureCliCredential
from azure.ai.ml.entities import (
    ManagedOnlineDeployment,
    ManagedOnlineEndpoint,
    Model,
    CodeConfiguration,
    Environment,
)

credential = AzureCliCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

workspace_location = ml_client.workspaces.get(workspace_name).location

## 2. Configure deployment

### 2.1 Define an endpoint configuration

In [ ]:
endpoint = ManagedOnlineEndpoint(name=endpoint_name, auth_mode="key")

### 2.2 Define a deployment configuration

In [ ]:
deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint_name,
    model=Model(path="../../model-1/model/"),
    code_configuration=CodeConfiguration(
        code="../../model-1/onlinescoring/", scoring_script="score_managedidentity.py"
    ),
    environment=Environment(
        conda_file="../../model-1/environment/conda-managedidentity.yml",
        image="mcr.microsoft.com/azureml/minimal-ubuntu22.04-py39-cpu-inference:latest",
    ),
    instance_type="Standard_DS3_v2",
    instance_count=1,
    environment_variables={
        "STORAGE_ACCOUNT_NAME": storage_account_name,
        "STORAGE_CONTAINER_NAME": storage_container_name,
        "FILE_NAME": file_name,
    },
)

## 3. Create the managed identity

With a system-assigned managed identity, Azure creates a managed identity on your behalf.

## 4. Create storage account and container

### 4.1 Get a handle to the `StorageManagementClient`

In [ ]:
from azure.mgmt.storage import StorageManagementClient
from azure.storage.blob import ContainerClient
from azure.mgmt.storage.models import Sku, StorageAccountCreateParameters, BlobContainer

credential = AzureCliCredential()
storage_client = StorageManagementClient(
    credential=credential, subscription_id=subscription_id
)

### 4.2 Define and create the storage account

In [ ]:
storage_account_parameters = StorageAccountCreateParameters(
    sku=Sku(name="Standard_LRS"), kind="Storage", location=workspace_location
)

storage_account = storage_client.storage_accounts.begin_create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    parameters=storage_account_parameters,
).result()

### 4.3 Create a blob container

In [ ]:
blob_container = storage_client.blob_containers.create(
    resource_group_name=resource_group,
    account_name=storage_account_name,
    container_name=storage_container_name,
    blob_container=BlobContainer(),
)

### 4.4 Get the storage account key and create a `ContainerClient`

In [ ]:
res = storage_client.storage_accounts.list_keys(
    resource_group_name=resource_group,
    account_name=storage_account_name,
)
key = res.keys[0].value

container_client = ContainerClient(
    account_url=storage_account.primary_endpoints.blob,
    container_name=storage_container_name,
    credential=key,
)

### 4.5 Upload a blob to the container

In [ ]:
with open(file_name, "rb") as f:
    container_client.upload_blob(name=file_name, data=f.read())

## 5. Create an online endpoint

### 5.1 Create the endpoint

In [ ]:
ml_client.online_endpoints.begin_create_or_update(endpoint).result()

### 5.2 Confirm identity details

In [ ]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
print(endpoint.identity.type)
print(endpoint.identity.principal_id)

## 6. Give access permission to the managed identity

### 6.1 Get an `AuthorizationManagementClient` to list Role Definitions

In [ ]:
from azure.mgmt.authorization import AuthorizationManagementClient
from azure.mgmt.authorization.v2018_01_01_preview.models import RoleDefinition
import uuid

role_definition_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2018-01-01-preview",
)

### 6.2 Get an `AuthorizationManagementClient` to make Role Assignments

In [ ]:
from azure.mgmt.authorization.v2020_10_01_preview.models import (
    RoleAssignment,
    RoleAssignmentCreateParameters,
)

role_assignment_client = AuthorizationManagementClient(
    credential=credential,
    subscription_id=subscription_id,
    api_version="2020-10-01-preview",
)

### 6.3 Get System-assigned identity details

In [ ]:
endpoint = ml_client.online_endpoints.get(endpoint_name)
system_principal_id = endpoint.identity.principal_id

### 6.4 Assign `Storage Blob Data Reader`

In [ ]:
role_name = "Storage Blob Data Reader"
scope = storage_account.id

role_defs = role_definition_client.role_definitions.list(scope=scope)
role_def = next((r for r in role_defs if r.role_name == role_name))

role_assignment_client.role_assignments.create(
    scope=scope,
    role_assignment_name=str(uuid.uuid4()),
    parameters=RoleAssignmentCreateParameters(
        role_definition_id=role_def.id, principal_id=system_principal_id
    ),
)

## 7. Create a deployment with your configuration

### 7.1 Create the deployment

In [ ]:
ml_client.online_deployments.begin_create_or_update(deployment).result()

### 7.2 Check the status of the deployment

In [ ]:
deployment = ml_client.online_deployments.get(
    endpoint_name=endpoint_name, name=deployment.name
)
print(deployment)

### 7.3 Get the deployment logs

In [ ]:
ml_client.online_deployments.get_logs(deployment.name, deployment.endpoint_name, 1000)

### 7.4 Set traffic to 100% for deployment

In [ ]:
endpoint.traffic = {str(deployment.name): 100}
ml_client.begin_create_or_update(endpoint).result()

## 8. Test the endpoint

In [ ]:
sample_data = "../../model-1/sample-request.json"
ml_client.online_endpoints.invoke(endpoint_name=endpoint_name, request_file=sample_data)

## 9. Delete the endpoint and storage account

### 9.1 Delete the endpoint

In [ ]:
ml_client.online_endpoints.begin_delete(endpoint_name)

### 9.2 Delete the storage account

In [ ]:
storage_client.storage_accounts.delete(
    resource_group_name=resource_group, account_name=storage_account_name
)